# BSM2 Flowsheet tutorial: 

### This demonstration will show 
- Part 1: how to build, initialize, and simulate the flowsheet for Benchmark Simulation Model 2 (BSM2). 
- Part 2: Demonstrate optimization and sensitivity analysis over specific parameters of the flowsheet (pending).
- Useful Links:
    - Public Github Repository: https://github.com/watertap-org/watertap
    - Documentation: https://watertap.readthedocs.io/en/stable/
    - Activated Sludge Model No. 1 (ASM1) documentation: https://watertap.readthedocs.io/en/stable/technical_reference/property_models/ASM1.html
    - Anaerobic Digestion Model No. 1 (ADM1) documentation: https://watertap.readthedocs.io/en/stable/technical_reference/property_models/ADM1.html
    - ASM1-ADM1 Translator documentation: https://watertap.readthedocs.io/en/stable/technical_reference/unit_models/translators/translator_asm1_adm1.html
    - ADM1-ASM1 Translator documentation: https://watertap.readthedocs.io/en/stable/technical_reference/unit_models/translators/translator_adm1_asm1.html    
    - Unit Model documentation: https://watertap.readthedocs.io/en/stable/technical_reference/unit_models/index.html
    - BSM2 flowsheet code: https://github.com/watertap-org/watertap/blob/main/watertap/examples/flowsheets/case_studies/full_water_resource_recovery_facility/BSM2.py


# Part 1: Build, setup, and simulate the Benchmark Simulation Model

<img src="BSM2.png" width="1000" height="680">

## Step 1: Import libraries from Pyomo, IDAES, and WaterTAP.

### Step 1.1: Import some essentials from Pyomo and IDAES:

In [ ]:
import pyomo.environ as pyo
from pyomo.network import Arc, SequentialDecomposition
from idaes.core import FlowsheetBlock
import idaes.logger as idaeslog
from idaes.core.solvers import get_solver
import idaes.core.util.scaling as iscale

### Step 1.2: Import unit models from WaterTAP and IDAES:

In [ ]:
# Import anaerobic digester model
from watertap.unit_models.anaerobic_digester import AD

# Import CSTR with oxygen injection model
from watertap.unit_models.cstr_injection import CSTR_Injection

# Import BSM2 separator models 
from watertap.unit_models.thickener import Thickener
from watertap.unit_models.dewatering import DewateringUnit

# Import idaes unit models for separators and mixers and ASM models
from idaes.models.unit_models import (
    CSTR,
    Feed,
    Mixer,
    Separator,
    PressureChanger,
    Product,
)
from idaes.models.unit_models.separator import SplittingType

# import translator models from WaterTAP
from watertap.unit_models.translators.translator_asm1_adm1 import Translator_ASM1_ADM1
from watertap.unit_models.translators.translator_adm1_asm1 import Translator_ADM1_ASM1

### Step 1.3: Import all BSM2 required property models

Property blocks are an important building block in WaterTap as they are a Python class which contain information on units, physical properties, etc.

In [ ]:
# Import Anaerobic Digester Model properties 
from watertap.property_models.anaerobic_digestion.adm1_properties import (
    ADM1ParameterBlock,
)
from watertap.property_models.anaerobic_digestion.adm1_reactions import (
    ADM1ReactionParameterBlock,
)
from watertap.property_models.anaerobic_digestion.adm1_properties_vapor import (
    ADM1_vaporParameterBlock,
)

# Import Activated Sludge Model properties 
from watertap.property_models.activated_sludge.asm1_properties import (
    ASM1ParameterBlock,
)
from watertap.property_models.activated_sludge.asm1_reactions import (
    ASM1ReactionParameterBlock,
)

## Step 2 Flowsheet building

## Step 2.1: Create Flowsheet
We will start by creating a pyomo model and a flowsheet

In [ ]:
m = pyo.ConcreteModel()

m.fs = FlowsheetBlock(dynamic=False)

We then include all the necessary property blocks we imported into the flowsheet. Namely, we include the ASM1 and ADM1 models, which are separated into their respective property and reaction models. Additionally, the vapor phase of ADM1 was separated into its own property model.

In [ ]:
m.fs.props_ASM1 = ASM1ParameterBlock()
m.fs.props_ADM1 = ADM1ParameterBlock()
m.fs.props_vap = ADM1_vaporParameterBlock()
m.fs.ADM1_rxn_props = ADM1ReactionParameterBlock(property_package=m.fs.props_ADM1)
m.fs.ASM1_rxn_props = ASM1ReactionParameterBlock(property_package=m.fs.props_ASM1)

### Step 2.2: Setup Activated Sludge process

We will start by setting up the activated sludge process unit models and connectivity.

First, we set up a Feed model for our feed stream and will name it `Feedwater`. 

In [ ]:
# Feed water stream
m.fs.FeedWater = Feed(property_package=m.fs.props_ASM1)
# Mixer for feed water and recycled sludge
m.fs.MX1 = Mixer(
    property_package=m.fs.props_ASM1, inlet_list=["feed_water", "recycle"]
)
# First reactor (anoxic) - standard CSTR
m.fs.R1 = CSTR(
    property_package=m.fs.props_ASM1, reaction_package=m.fs.ASM1_rxn_props
)
# Second reactor (anoxic) - standard CSTR
m.fs.R2 = CSTR(
    property_package=m.fs.props_ASM1, reaction_package=m.fs.ASM1_rxn_props
)
# Third reactor (aerobic) - CSTR with injection
m.fs.R3 = CSTR_Injection(
    property_package=m.fs.props_ASM1, reaction_package=m.fs.ASM1_rxn_props
)
# Fourth reactor (aerobic) - CSTR with injection
m.fs.R4 = CSTR_Injection(
    property_package=m.fs.props_ASM1, reaction_package=m.fs.ASM1_rxn_props
)
# Fifth reactor (aerobic) - CSTR with injection
m.fs.R5 = CSTR_Injection(
    property_package=m.fs.props_ASM1, reaction_package=m.fs.ASM1_rxn_props
)
m.fs.SP5 = Separator(
    property_package=m.fs.props_ASM1, outlet_list=["underflow", "overflow"]
)
# Clarifier
m.fs.CL1 = Separator(
    property_package=m.fs.props_ASM1,
    outlet_list=["underflow", "effluent"],
    split_basis=SplittingType.componentFlow,
)
# Sludge purge splitter
m.fs.SP6 = Separator(
    property_package=m.fs.props_ASM1,
    outlet_list=["recycle", "waste"],
    split_basis=SplittingType.totalFlow,
)
# Mixing sludge recycle and R5 underflow
m.fs.MX6 = Mixer(
    property_package=m.fs.props_ASM1, inlet_list=["clarifier", "reactor"]
)
# Product Blocks
m.fs.Treated = Product(property_package=m.fs.props_ASM1)
# Recycle pressure changer - use a simple isothermal unit for now
m.fs.P1 = PressureChanger(property_package=m.fs.props_ASM1)

Secondly we will use pyomo arcs as streams connecting unit to unit

In [ ]:
# Link units
m.fs.stream2 = Arc(source=m.fs.MX1.outlet, destination=m.fs.R1.inlet)
m.fs.stream3 = Arc(source=m.fs.R1.outlet, destination=m.fs.R2.inlet)
m.fs.stream4 = Arc(source=m.fs.R2.outlet, destination=m.fs.R3.inlet)
m.fs.stream5 = Arc(source=m.fs.R3.outlet, destination=m.fs.R4.inlet)
m.fs.stream6 = Arc(source=m.fs.R4.outlet, destination=m.fs.R5.inlet)
m.fs.stream7 = Arc(source=m.fs.R5.outlet, destination=m.fs.SP5.inlet)
m.fs.stream8 = Arc(source=m.fs.SP5.overflow, destination=m.fs.CL1.inlet)
m.fs.stream9 = Arc(source=m.fs.SP5.underflow, destination=m.fs.MX6.reactor)
m.fs.stream10 = Arc(source=m.fs.CL1.effluent, destination=m.fs.Treated.inlet)
m.fs.stream11 = Arc(source=m.fs.CL1.underflow, destination=m.fs.SP6.inlet)
m.fs.stream13 = Arc(source=m.fs.SP6.recycle, destination=m.fs.MX6.clarifier)
m.fs.stream14 = Arc(source=m.fs.MX6.outlet, destination=m.fs.P1.inlet)
m.fs.stream15 = Arc(source=m.fs.P1.outlet, destination=m.fs.MX1.recycle)
pyo.TransformationFactory("network.expand_arcs").apply_to(m)

Next we will set the conditions for the inlet water

In [ ]:
# Feed Water Conditions
m.fs.FeedWater.flow_vol.fix(20648 * pyo.units.m**3 / pyo.units.day)
m.fs.FeedWater.temperature.fix(308.15 * pyo.units.K)
m.fs.FeedWater.pressure.fix(1 * pyo.units.atm)
m.fs.FeedWater.conc_mass_comp[0, "S_I"].fix(
    27.2262 * pyo.units.g / pyo.units.m**3
)
m.fs.FeedWater.conc_mass_comp[0, "S_S"].fix(
    58.1762 * pyo.units.g / pyo.units.m**3
)
m.fs.FeedWater.conc_mass_comp[0, "X_I"].fix(92.499 * pyo.units.g / pyo.units.m**3)
m.fs.FeedWater.conc_mass_comp[0, "X_S"].fix(
    363.9435 * pyo.units.g / pyo.units.m**3
)
m.fs.FeedWater.conc_mass_comp[0, "X_BH"].fix(
    50.6833 * pyo.units.g / pyo.units.m**3
)
m.fs.FeedWater.conc_mass_comp[0, "X_BA"].fix(0 * pyo.units.g / pyo.units.m**3)
m.fs.FeedWater.conc_mass_comp[0, "X_P"].fix(0 * pyo.units.g / pyo.units.m**3)
m.fs.FeedWater.conc_mass_comp[0, "S_O"].fix(0 * pyo.units.g / pyo.units.m**3)
m.fs.FeedWater.conc_mass_comp[0, "S_NO"].fix(0 * pyo.units.g / pyo.units.m**3)
m.fs.FeedWater.conc_mass_comp[0, "S_NH"].fix(
    23.8595 * pyo.units.g / pyo.units.m**3
)
m.fs.FeedWater.conc_mass_comp[0, "S_ND"].fix(
    5.6516 * pyo.units.g / pyo.units.m**3
)
m.fs.FeedWater.conc_mass_comp[0, "X_ND"].fix(
    16.1298 * pyo.units.g / pyo.units.m**3
)
m.fs.FeedWater.alkalinity.fix(7 * pyo.units.mol / pyo.units.m**3)

Finally for the activated sludge models we will set the conditions for the unit models. We will start with the reactors volume

In [ ]:
# Reactor sizing
m.fs.R1.volume.fix(1500 * pyo.units.m**3)
m.fs.R2.volume.fix(1500 * pyo.units.m**3)
m.fs.R3.volume.fix(3000 * pyo.units.m**3)
m.fs.R4.volume.fix(3000 * pyo.units.m**3)
m.fs.R5.volume.fix(3000 * pyo.units.m**3)

The oxygen concentration in reactors 3-5 is injected through the CSTR injection reactors

In [ ]:
# Injection rates to Reactions 3, 4 and 5
for j in m.fs.props_ASM1.component_list:
    if j != "S_O":
        # All components except S_O have no injection
        m.fs.R3.injection[:, :, j].fix(0)
        m.fs.R4.injection[:, :, j].fix(0)
        m.fs.R5.injection[:, :, j].fix(0)
# Then set injections rates for O2
m.fs.R3.outlet.conc_mass_comp[:, "S_O"].fix(0.46635e-3)
m.fs.R4.outlet.conc_mass_comp[:, "S_O"].fix(1.4284e-3)
m.fs.R5.outlet.conc_mass_comp[:, "S_O"].fix(1.3748e-3)

Then we set up the separation, going through the splitters and the clarifier

In [ ]:
# Set fraction of outflow from reactor 5 that goes to recycle
m.fs.SP5.split_fraction[:, "underflow"].fix(0.6)

# Clarifier
m.fs.CL1.split_fraction[0, "effluent", "H2O"].fix(20640.7791 / (20640.7791 + 20648))
m.fs.CL1.split_fraction[0, "effluent", "S_I"].fix(20640.7791 / (20640.7791 + 20648))
m.fs.CL1.split_fraction[0, "effluent", "S_S"].fix(20640.7791 / (20640.7791 + 20648))
m.fs.CL1.split_fraction[0, "effluent", "X_I"].fix(
    5.9191 * 20640.7791 / (5.9191 * 20640.7791 + 3036.2175 * 20648)
)
m.fs.CL1.split_fraction[0, "effluent", "X_S"].fix(
    0.12329 * 20640.7791 / (0.12329 * 20640.7791 + 63.2392 * 20648)
)
m.fs.CL1.split_fraction[0, "effluent", "X_BH"].fix(0.00193)
m.fs.CL1.split_fraction[0, "effluent", "X_BA"].fix(0.00193)
m.fs.CL1.split_fraction[0, "effluent", "X_P"].fix(0.00193)
m.fs.CL1.split_fraction[0, "effluent", "S_O"].fix(20640.7791 / (20640.7791 + 20648))
m.fs.CL1.split_fraction[0, "effluent", "S_NO"].fix(
    20640.7791 / (20640.7791 + 20648)
)
m.fs.CL1.split_fraction[0, "effluent", "S_NH"].fix(
    20640.7791 / (20640.7791 + 20648)
)
m.fs.CL1.split_fraction[0, "effluent", "S_ND"].fix(
    20640.7791 / (20640.7791 + 20648)
)
m.fs.CL1.split_fraction[0, "effluent", "X_ND"].fix(0.00193)
m.fs.CL1.split_fraction[0, "effluent", "S_ALK"].fix(
    20640.7791 / (20640.7791 + 20648)
)

# Sludge purge separator
m.fs.SP6.split_fraction[:, "recycle"].fix(20648 / 20948)

The last thing required is the pressure for the recycle pump

In [ ]:
# Outlet pressure from recycle pump
m.fs.P1.outlet.pressure.fix(101325)

### Step 2.2: Setup Anaerobic digester process

We will start by setting up the anaerobic digester process unit models and connectivity.

First, like above, we set up unit models. specifically the reactor. Which will have two different property blocks as it has two separate phases liquid and gas

In [ ]:
m.fs.RADM = AD(
    liquid_property_package=m.fs.props_ADM1,
    vapor_property_package=m.fs.props_vap,
    reaction_package=m.fs.ADM1_rxn_props,
    has_heat_transfer=True,
    has_pressure_change=False,
)

In order to connect the ADM and ASM models translator blocks are required as they track different species

In [ ]:
m.fs.asm_adm = Translator_ASM1_ADM1(
    inlet_property_package=m.fs.props_ASM1,
    outlet_property_package=m.fs.props_ADM1,
    reaction_package=m.fs.ADM1_rxn_props,
    has_phase_equilibrium=False,
    outlet_state_defined=True,
)

m.fs.adm_asm = Translator_ADM1_ASM1(
    inlet_property_package=m.fs.props_ADM1,
    outlet_property_package=m.fs.props_ASM1,
    reaction_package=m.fs.ADM1_rxn_props,
    has_phase_equilibrium=False,
    outlet_state_defined=True,
)

We the set up the separators and mixers 

In [ ]:
m.fs.CL = Separator(
    property_package=m.fs.props_ASM1,
    outlet_list=["underflow", "effluent"],
    split_basis=SplittingType.componentFlow,
)

m.fs.TU = Thickener(property_package=m.fs.props_ASM1)
m.fs.DU = DewateringUnit(property_package=m.fs.props_ASM1)

m.fs.MX2 = Mixer(
    property_package=m.fs.props_ASM1, inlet_list=["feed_water1", "recycle1"]
)
m.fs.MX3 = Mixer(
    property_package=m.fs.props_ASM1, inlet_list=["feed_water2", "recycle2"]
)
m.fs.MX4 = Mixer(
    property_package=m.fs.props_ASM1, inlet_list=["thickener", "clarifier"]
)

We add operating conditions to the unit models. We start with the primary clarifier

In [ ]:
# Clarifier
m.fs.CL.split_fraction[0, "effluent", "H2O"].fix(0.993)
m.fs.CL.split_fraction[0, "effluent", "S_I"].fix(0.993)
m.fs.CL.split_fraction[0, "effluent", "S_S"].fix(0.993)
m.fs.CL.split_fraction[0, "effluent", "X_I"].fix(0.5192)
m.fs.CL.split_fraction[0, "effluent", "X_S"].fix(0.5192)
m.fs.CL.split_fraction[0, "effluent", "X_BH"].fix(0.5192)
m.fs.CL.split_fraction[0, "effluent", "X_BA"].fix(0.5192)
m.fs.CL.split_fraction[0, "effluent", "X_P"].fix(0.5192)
m.fs.CL.split_fraction[0, "effluent", "S_O"].fix(0.993)
m.fs.CL.split_fraction[0, "effluent", "S_NO"].fix(0.993)
m.fs.CL.split_fraction[0, "effluent", "S_NH"].fix(0.993)
m.fs.CL.split_fraction[0, "effluent", "S_ND"].fix(0.993)
m.fs.CL.split_fraction[0, "effluent", "X_ND"].fix(0.5192)
m.fs.CL.split_fraction[0, "effluent", "S_ALK"].fix(0.993)

We then set up the anaerobic digester operating conditions

In [ ]:
# Anaerobic digester
m.fs.RADM.volume_liquid.fix(3400)
m.fs.RADM.volume_vapor.fix(300)
m.fs.RADM.liquid_outlet.temperature.fix(308.15)

Additionally, the dewatering unit includes an equation relating its hydraulic retention time to its volume and influent flowrate. We can choose to specify hydraulic retention time or the unit's volume to satisfy 0 degrees of freedom.

In [ ]:
# Dewatering unit
m.fs.DU.hydraulic_retention_time.fix(1800 * pyo.units.s)

Similarly, the thickener unit includes the same equation, as well as an equation relating the thickener's dimensions. Here, we fix hydraulic retention time and thickener diameter to satisfy 0 degrees of freedom.

In [ ]:
# Thickener unit
m.fs.TU.hydraulic_retention_time.fix(86400 * pyo.units.s)
m.fs.TU.diameter.fix(10 * pyo.units.m)

We then again add arcs as streams linking the unit models

In [ ]:
m.fs.stream2adm = Arc(
    source=m.fs.RADM.liquid_outlet, destination=m.fs.adm_asm.inlet
)
m.fs.stream6adm = Arc(source=m.fs.SP6.waste, destination=m.fs.TU.inlet)
m.fs.stream3adm = Arc(source=m.fs.TU.underflow, destination=m.fs.MX4.thickener)
m.fs.stream7adm = Arc(source=m.fs.TU.overflow, destination=m.fs.MX3.recycle2)
m.fs.stream9adm = Arc(source=m.fs.CL.underflow, destination=m.fs.MX4.clarifier)
m.fs.stream4adm = Arc(source=m.fs.adm_asm.outlet, destination=m.fs.DU.inlet)
m.fs.stream5adm = Arc(source=m.fs.DU.overflow, destination=m.fs.MX2.recycle1)
m.fs.stream01 = Arc(source=m.fs.FeedWater.outlet, destination=m.fs.MX2.feed_water1)
m.fs.stream02 = Arc(source=m.fs.MX2.outlet, destination=m.fs.MX3.feed_water2)
m.fs.stream03 = Arc(source=m.fs.MX3.outlet, destination=m.fs.CL.inlet)
m.fs.stream04 = Arc(source=m.fs.CL.effluent, destination=m.fs.MX1.feed_water)
m.fs.stream10adm = Arc(source=m.fs.MX4.outlet, destination=m.fs.asm_adm.inlet)
m.fs.stream1adm = Arc(source=m.fs.asm_adm.outlet, destination=m.fs.RADM.inlet)
pyo.TransformationFactory("network.expand_arcs").apply_to(m)

In [ ]:
# Apply scaling
iscale.calculate_scaling_factors(m.fs)

# Step 3: Solve the square problem
## Step 3.1: Initialize the model

In [ ]:
# Initialize flowsheet
# Apply sequential decomposition - 1 iteration should suffice
seq = SequentialDecomposition()
# seq.options.select_tear_method = "heuristic"
seq.options.tear_method = "Direct"
seq.options.iterLim = 1
seq.options.tear_set = [m.fs.stream2, m.fs.stream10adm]

G = seq.create_graph(m)
# Uncomment this code to see tear set and initialization order
order = seq.calculation_order(G)
print("Initialization Order")
for o in order:
    print(o[0].name)

# Initial guesses for flow into first reactor
tear_guesses1 = {
    "flow_vol": {0: 103531 / 24 / 3600},
    "conc_mass_comp": {
        (0, "S_I"): 0.028,
        (0, "S_S"): 0.012,
        (0, "X_I"): 1.532,
        (0, "X_S"): 0.069,
        (0, "X_BH"): 2.233,
        (0, "X_BA"): 0.167,
        (0, "X_P"): 0.964,
        (0, "S_O"): 0.0011,
        (0, "S_NO"): 0.0073,
        (0, "S_NH"): 0.0072,
        (0, "S_ND"): 0.0016,
        (0, "X_ND"): 0.0040,
    },
    "alkalinity": {0: 0.0052},
    "temperature": {0: 308.15},
    "pressure": {0: 101325},
}

tear_guesses2 = {
    "flow_vol": {0: 170 / 24 / 3600},
    "conc_mass_comp": {
        (0, "S_I"): 0.028,
        (0, "S_S"): 0.048,
        (0, "X_I"): 10.362,
        (0, "X_S"): 20.375,
        (0, "X_BH"): 10.210,
        (0, "X_BA"): 0.553,
        (0, "X_P"): 3.204,
        (0, "S_O"): 0.00025,
        (0, "S_NO"): 0.00169,
        (0, "S_NH"): 0.0289,
        (0, "S_ND"): 0.00468,
        (0, "X_ND"): 0.906,
    },
    "alkalinity": {0: 0.00715},
    "temperature": {0: 308.15},
    "pressure": {0: 101325},
}

# Pass the tear_guess to the SD tool
seq.set_guesses_for(m.fs.R1.inlet, tear_guesses1)
seq.set_guesses_for(m.fs.asm_adm.inlet, tear_guesses2)

We then run the initialization by creating a function to initialize each unit model and running it

In [ ]:
def function(unit):
    unit.initialize(outlvl=idaeslog.INFO_HIGH)

seq.run(m, function)

## Step 3.2: Run solver
Solve the model by running the flowsheet using the ipopt solver.

In [ ]:
solver = get_solver()
results = solver.solve(m, tee=True)

We run an assertion to make sure the solver found the optimal solution

In [ ]:
pyo.assert_optimal_termination(results)

## Step 3.3: report solution
we then report the treated water block

In [ ]:
m.fs.Treated.report()

# Part 2: Demonstrate optimization and sensitivity analysis over specific parameters of the flowsheet (pending)
The addition of unit capital costs and O&M costs is underway. Subsequently, an example of cost optimization of the steady-state BSM2 flowsheet will be presented.